# NLP

El Procesamiento de Lenguaje Natural (NLP, por sus siglas en inglés) es una subárea de la inteligencia artificial que se centra en la interacción entre las computadoras y el lenguaje humano. Su aplicación en el contexto médico, especialmente en el diagnóstico con errores gramaticales, puede ofrecer diversos beneficios:

1. **Corrección Automática de Errores:** Una de las aplicaciones más directas del NLP es la corrección de errores gramaticales y ortográficos. Herramientas como TextBlob, pueden ayudar a identificar y corregir errores en el texto, aunque no son infalibles y a veces pueden introducir errores.

2. **Estandarización de Terminología:** En el ámbito médico, la terminología es crucial. Un término mal empleado o escrito incorrectamente puede cambiar significativamente el significado. El NLP puede ayudar a mapear términos mal escritos o variaciones de un término a una forma estándar.

3. **Extracción de Información:** El NLP permite extraer información relevante de textos desestructurados. Por ejemplo, si un médico escribe notas sobre un paciente, el NLP puede extraer detalles como síntomas, medicamentos prescritos y otros datos relevantes.

4. **Aumento de la Eficiencia:** Automatizar el proceso de revisión y corrección de diagnósticos mediante NLP puede ahorrar tiempo y reducir la carga de trabajo de los profesionales médicos.

5. **Mejora en la Calidad de la Atención Médica:** Un diagnóstico claro y preciso es fundamental para un tratamiento eficaz. Al reducir errores y ambigüedades en el diagnóstico, el NLP puede contribuir a una mejor atención al paciente.

6. **Minimizar Riesgos:** Errores en el diagnóstico pueden llevar a tratamientos incorrectos o a malentendidos que pueden ser perjudiciales para el paciente. Al corregir estos errores, se reduce el riesgo de posibles complicaciones.

7. **Análisis de Sentimiento y Detección de Urgencia:** El NLP puede ser utilizado para analizar el sentimiento detrás de las palabras o para detectar casos que requieran atención urgente basándose en las palabras utilizadas.

8. **Integración con Sistemas de Aprendizaje Automático:** Una vez que los datos estén limpios y estructurados, pueden ser alimentados en sistemas de aprendizaje automático para hacer predicciones o identificar patrones en los diagnósticos.

Sin embargo, es importante señalar que, aunque el NLP tiene un potencial considerable, su aplicación en el contexto médico también tiene desafíos. El lenguaje médico es complejo y a menudo requiere una comprensión profunda del contexto. Además, un error en la corrección o interpretación del NLP puede tener consecuencias graves en un entorno médico. Por lo tanto, es crucial que cualquier sistema de NLP implementado en este ámbito sea exhaustivamente probado y validado.

In [3]:
# Modulos
import pandas as pd
import numpy as np

import collections
import time

import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from textblob import TextBlob

from spellchecker import SpellChecker
import spacy
from spacy.language import Language


In [2]:
%load_ext kedro.ipython

[09/17/23 01:59:31] INFO     Resolved project path as: c:\Users\Pablo                               __init__.py:136
                             Villar\Desktop\CURSOS\KSchool\Máster en Data                                          
                             Science\TFM\nuevo_entorno_kedro\hado.                                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[09/17/23 01:59:32] WARNING  C:\Users\Pablo                                                         warnings.py:109
                             Villar\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.                
                             py:126: DeprecationWarning: `kedro.extras.datasets` is deprecated and                 
                             will be removed in Kedro 0.19, install `kedro-datasets` instead by                    
                             running `pip install kedro-datasets`.                                                 
                               return _bootstrap._gcd_import(name[level:], package, level)                         
                                                                                                                   

[09/17/23 01:59:34] INFO     Kedro project hado                                                     __init__.py:105

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:106
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        __init__.py:112

In [21]:
df = catalog.load("hado_final")

[09/17/23 02:24:41] INFO     Loading data from 'hado_final' (CSVDataSet)...                     data_catalog.py:345

Una vez realizada la limpieza y preprocesamiento
## Hacemos la vectorización de texto

La vectorización de texto es el proceso de convertir palabras en vectores numéricos que pueden ser entendidos y procesados por un algoritmo de machine learning. Existen diferentes técnicas para hacer esto, entre las que se incluyen Bag of Words, TF-IDF, y Word2Vec.

>Word2Vec es un método que utiliza redes neuronales para aprender representaciones vectoriales de palabras a partir de un corpus de texto. Este método tiene la ventaja de que puede capturar el significado semántico de las palabras.

Tenemos que convertir los diagnosticos en una lista de listas haciendo un split de la variable 
```python
diagnosticos = df['diagnosticos'].unique()`
```

In [42]:
# Cargamos las stopwords

# import nltk
# from nltk.corpus import stopwords

# Descargar las stopwords de NLTK
nltk.download('stopwords')

# Obtener las stopwords en español
stopwords_es = stopwords.words('spanish')

# Aplicar la eliminación de stopwords a tu columna de texto
df['diagnostico'] = df['diagnostico'].apply(lambda x: ' '.join([word for word in x.split()\
                                                                if word not in (stopwords_es)]))


[nltk_data] Downloading package stopwords to C:\Users\Pablo
[nltk_data]     Villar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
diagnosticos = df['diagnostico'].unique()
diagnosticos_split = [diagnostico.split() for diagnostico in diagnosticos]

In [44]:
diagnosticos_split[:10]

[['cancer', 'orl'],
 ['melanoma', 'vulvar'],
 ['cancer', 'broncogenico'],
 ['cirrosis', 'hepatica'],
 ['neoplasia', 'mama', 'estadio', 'IV'],
 ['anemia'],
 ['adenoma', 'pulmon'],
 ['leucemia', 'linfatica', 'cronica'],
 ['fx', 'meseta', 'tibial'],
 ['cancer', 'mama']]

In [45]:
from gensim.models import Word2Vec

# Asumiendo que tus diagnósticos ya están preprocesados y almacenados en una lista de listas,
# donde cada lista interna contiene las palabras de un diagnóstico.
# Por ejemplo: [["neoplasia", "pancreas"], ["adenocarcinoma", "de", "pulmón"], ...]

# Entrenamos el modelo Word2Vec
model = Word2Vec(diagnosticos_split, min_count=1, vector_size=50)

# Ahora podemos obtener el vector de una palabra en particular
vector = model.wv['cancer']

# O incluso obtener la palabra más similar a 'cancer'
similar = model.wv.most_similar('cancer')


In [46]:
vector

array([-0.00206307, -0.00134983,  0.01014814,  0.01863859, -0.02026581,
       -0.01824549,  0.01603529,  0.02281343, -0.01487547, -0.00801561,
        0.01485418, -0.00788157, -0.00617845,  0.01374094, -0.01113503,
       -0.00174002,  0.00827018,  0.00190461, -0.02161521, -0.02260242,
        0.01620434,  0.01315533,  0.01823759, -0.00120759,  0.01599858,
       -0.00570859, -0.00510819,  0.01146678, -0.01847552, -0.00714802,
       -0.01240762, -0.00207771,  0.01616402, -0.01300409, -0.00878509,
       -0.00353266,  0.01858204, -0.0101256 ,  0.00163351, -0.01220511,
       -0.01589034,  0.00937978, -0.01804782, -0.00937661,  0.00864735,
        0.00038583, -0.0168338 ,  0.01493173,  0.01281624,  0.0198029 ],
      dtype=float32)

In [47]:
similar

[('septico', 0.5110003352165222),
 ('fracturas', 0.4618563950061798),
 ('isquemicaareumatoide', 0.4074855446815491),
 ('adenocarcino', 0.39057669043540955),
 ('denencia', 0.37653809785842896),
 ('trigono', 0.37424391508102417),
 ('global', 0.3738190531730652),
 ('fragil', 0.3601739704608917),
 ('reumatoide', 0.35848185420036316),
 ('broncoaspiraciondeterioro', 0.357061505317688)]

### En el código anterior, min_count es el número mínimo de veces que una palabra debe aparecer en el corpus para ser incluida en el modelo, y vector_size es la dimensión de los vectores de palabra.

Word2Vec(diagnosticos_split, min_count=1, vector_size=50): Estás entrenando un modelo Word2Vec con tus diagnósticos. La opción min_count=1 significa que las palabras que aparecen al menos una vez en tus diagnósticos serán consideradas por el modelo, y vector_size=50 define la dimensionalidad de los vectores de palabras generados por el modelo.

model.wv['cancer']: Esto te devuelve el vector de la palabra 'cancer'. Este vector es un arreglo de 50 números (porque has definido vector_size=50) que representa el significado de 'cancer' según lo aprendido por el modelo de tus diagnósticos.

model.wv.most_similar('cancer'): Esto te devuelve las palabras más similares a 'cancer' según lo aprendido por el modelo de tus diagnósticos. La similitud se mide en términos de la similitud del coseno entre los vectores de palabras.

Por favor, ten en cuenta que el modelo Word2Vec necesita una gran cantidad de datos para aprender representaciones de palabras significativas. Si solo tienes un pequeño número de diagnósticos, los vectores de palabras generados por el modelo podrían no ser muy significativos. En este caso, puedes considerar el uso de vectores de palabras pre-entrenados, como los disponibles en bibliotecas como spaCy o gensim.

In [48]:
# Filtramos los diagnósticos vacíos
diagnosticos_split = [diagnostico for diagnostico in diagnosticos_split if diagnostico]

# Luego, volvemos a calcular los vectores de diagnósticos
diagnosticos_vec = [np.mean([model.wv[word] for word in diagnostico if word in model.wv.key_to_index], axis=0) for diagnostico in diagnosticos_split]

In [49]:
from sklearn.cluster import KMeans

# Primero, necesitamos convertir nuestros diagnósticos a vectores.
# Podemos hacer esto promediando los vectores de todas las palabras en cada diagnóstico.
diagnosticos_vec = [np.mean([model.wv[word] for word in diagnostico], axis=0) for diagnostico in diagnosticos_split]

# Entonces podemos entrenar un modelo KMeans en estos vectores.
kmeans = KMeans(n_clusters=5)  # Ajusta este número a la cantidad de grupos que creas adecuada
kmeans.fit(diagnosticos_vec)

# Ahora podemos obtener las etiquetas de los grupos para cada diagnóstico
labels = kmeans.labels_

[09/16/23 21:14:11] WARNING  c:\Users\Pablo Villar\Desktop\CURSOS\KSchool\Máster en Data            warnings.py:109
                             Science\TFM\nuevo_entorno_kedro\lib\site-packages\sklearn\cluster\_kme                
                             ans.py:870: FutureWarning: The default value of `n_init` will change                  
                             from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to                     
                             suppress the warning                                                                  
                               warnings.warn(                                                                      
                                                                                                                   

In [50]:
labels

array([0, 1, 0, ..., 2, 4, 1])

In [51]:
# Crea un diccionario donde las llaves son las etiquetas del cluster y los valores son listas de diagnósticos pertenecientes a ese cluster
clusters = {i: [] for i in range(5)}  # Asegúrate de cambiar el rango para que coincida con el número de clusters que estás usando

for diagnostico, label in zip(diagnosticos_split, labels):
    clusters[label].append(' '.join(diagnostico))

# Ahora puedes imprimir los diagnósticos en cada cluster
for label, diagnosticos in clusters.items():
    print(f'Cluster {label}:')
    for diagnostico in diagnosticos[:10]:
        print(f'  {diagnostico}')


Cluster 0:
  cancer orl
  cancer broncogenico
  cancer mama
  cancer prostata
  cancer colon
  cancer pancreas
  cancer timo
  cancer parotida
  cancer endometrio
  cancer suprarrenal
Cluster 1:
  melanoma vulvar
  cirrosis hepatica
  adenoma pulmon
  tumor cerebral
  sarcoidosis
  bocio intratoracico
  intolerancia oral
  esclerosis multiple
  demencia fallo multiorganico
  fallo cardiaco
Cluster 2:
  neumonia
  niemannpick
  infeccion urinaria fallo cardiaco
  infeccion respiratoria
  sepsis respiratoria
  infeccion urinaria
  ITU
  smd
  amputacion
  acv
Cluster 3:
  neoplasia mama estadio IV
  adenoma colon IV
  cancer colon estadio IV
  miocardiopatia hipertrofica
  tratamiento antibiotico IV
  cancer lengua estadio IV
  cancer pulmon estadio IV
  adenocarcino colon estadio IV
  neo colon
  adenocarcinoma ovario estadio IV
Cluster 4:
  anemia
  leucemia linfatica cronica
  fx meseta tibial
  deterioro general
  melanoma fosa nasal
  sdown
  ulceras mmii
  fractura cadera
  fractur

In [52]:
for diagnostico, label in zip(diagnosticos_split, labels):
    clusters[label].append(' '.join(diagnostico))
    
    '''Está iterando sobre los diagnósticos y sus etiquetas de cluster correspondientes. 
    Para cada diagnóstico, transforma la lista de palabras en una cadena de texto (usando ' '.join(diagnostico)) 
    y la añade a la lista correspondiente en el diccionario clusters.
    
    Por ejemplo, si hay un diagnóstico que es ["neoplasia", "pancreas"] y su etiqueta de cluster es 2,
    este diagnóstico se transformaría en la cadena de texto "neoplasia pancreas" y se añadiría a la lista de diagnósticos
    del cluster 2 en el diccionario clusters. Así, al final, se crea un diccionario donde para cada cluster, 
    existe una lista de todos los diagnósticos (como cadenas de texto) que pertenecen a ese cluster'''

También puedes querer investigar las palabras más representativas de cada cluster. Esto puede darte una idea de las características que el modelo está utilizando para distinguir entre los diferentes clusters.
* Código de ejemplo para hacer eso:

In [53]:
import collections
# Para cada cluster, encuentra las 10 palabras más comunes
for label, diagnosticos in clusters.items():
    # Juntamos todos los diagnósticos en una sola lista de palabras
    palabras = [palabra for diagnostico in diagnosticos for palabra in diagnostico.split()]
    
    # Contamos las ocurrencias de cada palabra
    conteo_palabras = collections.Counter(palabras)
    
    print(f'Cluster {label}:')
    for palabra, conteo in conteo_palabras.most_common(10):
        print(f'  {palabra}: {conteo}')


Cluster 0:
  cancer: 266
  epidermoide: 28
  colon: 26
  pulmon: 20
  gastrico: 18
  prostata: 14
  vejiga: 10
  esofago: 8
  urotelial: 8
  hepaticas: 8
Cluster 1:
  neoplasia: 74
  carcinoma: 50
  cirrosis: 28
  colon: 26
  adenocarcinoma: 26
  absceso: 24
  pulmon: 20
  demencia: 20
  epoc: 20
  origen: 20
Cluster 2:
  infeccion: 160
  respiratoria: 50
  ITU: 46
  urinaria: 36
  sindrome: 26
  neumonia: 22
  celulitis: 22
  recto: 18
  cancer: 16
  herida: 14
Cluster 3:
  IV: 274
  estadio: 260
  cancer: 100
  neoplasia: 42
  colon: 38
  pulmon: 34
  neo: 34
  adenocarcinoma: 28
  prostata: 18
  recto: 18
Cluster 4:
  insuficiencia: 142
  cardiaca: 104
  cronica: 92
  cognitivo: 78
  deterioro: 76
  ulcera: 62
  ulceras: 60
  fragil: 48
  aguda: 40
  demencia: 38


>Estos clusters pueden ser útiles para entender qué tipos de diagnósticos se agrupan juntos y podrían ayudar en tareas como el análisis de datos, la recomendación de tratamientos, o la identificación de patrones en los diagnósticos.

- Es importante recordar que estos clusters se basan en la similitud de las palabras en los diagnósticos, que no necesariamente reflejan la similitud en las condiciones médicas que representan. Por lo tanto, cualquier interpretación o aplicación de estos clusters debe hacerse con cuidado y, preferiblemente, con la validación de un experto médico.

<hr>

# Otros

## Transformación diagnosticos

### 1.Corrección ortográfica: 
Usar bibliotecas como pyspellchecker para la corrección ortográfica. Sin embargo, este proceso puede ser computacionalmente costoso y no siempre proporciona resultados perfectos, ya que la corrección ortográfica puede ser un problema muy complejo. 

Otro enfoque podría ser construir un diccionario de correcciones comunes específicas para el conjunto de datos y aplicar estas correcciones de forma manual.

### 2.Procesamiento en lotes: 
En lugar de procesar el texto de cada fila de forma individual, puedes utilizar nlp.pipe(), que es una función de SpaCy que permite procesar textos en lotes. Esto puede ser mucho más eficiente que procesar cada texto de forma individual. 
> Pero queda descartado porque haré las correcciones de forma manual

### 3.Paralelización: 
Si estás utilizando una máquina con varios núcleos de CPU, puedes paralelizar el procesamiento para hacer un uso más eficiente de tus recursos computacionales. SpaCy tiene soporte para procesamiento paralelo a través de la función nlp.pipe(), simplemente estableciendo el argumento n_process a un número mayor que 1.

### 4.Clasificación de texto: 
Una vez corregidos los errores ortográficos. Se pasa a usar técnicas de aprendizaje automático para clasificar los diagnósticos en categorías. SpaCy tiene un componente de clasificación de texto con el que puedes entrenar los datos. Alternativamente, podrían usarse otras bibliotecas de aprendizaje automático, como scikit-learn, para construir un modelo de clasificación de texto.
> Utilizaré scikit-learn

### 5.Reducción de dimensionalidad: 
Algunas técnicas, como TF-IDF o word embeddings (por ejemplo, Word2Vec, FastText o las incrustaciones de palabras de SpaCy), pueden ayudarte a reducir la dimensionalidad de los datos de texto y a extraer características útiles para la clasificación.

### 6.Entrenamiento incremental: 
Si los datos están creciendo con el tiempo, puede considerarse el uso de un modelo que permita el entrenamiento incremental, es decir, un modelo que pueda ser actualizado con nuevos datos sin tener que ser reentrenado desde cero. Algunos modelos de SpaCy admiten esto, al igual que algunos modelos en bibliotecas como scikit-learn.

In [54]:
# Hacemos un sample
df_sample = df.sample(300)

In [55]:
df_sample.shape

(300, 50)

In [56]:
df_sample.head()

,h_procedencia,hospital_category,s_procedencia,procedencia_category,diagnostico,diagnosis_category,motivo_ing,ingreso_category,motivo_alta,alta_category,...,categorized_combined_otros,tiene_sedacion,morfina,midazolam,buscapina,haloperidol,levomepromazina,medico,ayuntamiento,year
117,clinico,Santiago,oncologia,Oncologia,cancer colon,Canceres y neoplasias,control sintomas,Sintomas,exitus,Exitus,...,Sintomas generales,0,0,0,0,0,0,suarez,desconocido,2017
2768,conxo,Santiago,upal,Unidad Paliativos,cancer colon,Canceres y neoplasias,control sintomas,Sintomas,exitus,Exitus,...,Desconocido/No especificado,1,0,0,0,0,0,villar del castillo,santiago,2021
374,no,no,no,Otros,tumor malar,Canceres y neoplasias,control sintomas,Sintomas,exitus,Exitus,...,Desconocido/No especificado,1,1,1,1,0,0,villar del castillo,desconocido,2017
2376,conxo,Santiago,cirugia maxilofacial,Otros,cancer lengua estadio IV,Canceres y neoplasias,control lesion mandibula,Evaluaciones,fin cuidados,Otros,...,Desconocido/No especificado,0,0,0,0,0,0,lopez renedo,oroso,2021
552,gil casares,Santiago,upal,Unidad Paliativos,cancer pulmon,Pulmonares y respiratorias,control dolor,Sintomas,exitus,Exitus,...,Complicaciones,1,1,1,1,0,1,ibanez alonso,desconocido,2017


# TextBlob

* Corregir errores ortográficos (usando la biblioteca TextBlob para corrección automática):

>OJO no puede tener valores NA o de otro tipo que no sea object

In [57]:
from textblob import TextBlob
import time

l = df_sample['diagnostico']

start_time = time.time()

for x in l[:10]:
    print(f"Diagnóstico original: {x}")
    print(f"Diagnóstico corregido: {TextBlob(x).correct()}")
    print("="*25)

end_time = time.time()
execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time} segundos")
    

Diagnóstico original: cancer colon
Diagnóstico corregido: cancer colon
Diagnóstico original: cancer colon
Diagnóstico corregido: cancer colon
Diagnóstico original: tumor malar
Diagnóstico corregido: tumor makar
Diagnóstico original: cancer lengua estadio IV
Diagnóstico corregido: cancer length studio of
Diagnóstico original: cancer pulmon
Diagnóstico corregido: cancer summon
Diagnóstico original: cancer ovarico estadio IV
Diagnóstico corregido: cancer ovarian studio of
Diagnóstico original: demencaparkinson
Diagnóstico corregido: demencaparkinson
Diagnóstico original: ulcera pierna
Diagnóstico corregido: ulcer pierre
Diagnóstico original: anciana fragil
Diagnóstico corregido: ancient frail
Diagnóstico original: infeccion respiratoria
Diagnóstico corregido: infection respiratory
Tiempo de ejecución: 1.2249009609222412 segundos


In [62]:
def correct_spelling_col(df, column):
    if df[column].dtype == 'object':  # comprueba si la columna es de tipo 'object'
        df[column] = df[column].apply(lambda x: str(TextBlob(x).correct()))

In [63]:
%%time 

correct_spelling_col(df_sample, 'diagnostico')

CPU times: total: 22.3 s
Wall time: 57.2 s


In [34]:
df_sample['diagnostico']

2203                              cancer varix
1903             ITU (infection tract urinary)
3245                            diverticulitis
2621                         ulcers isquemicas
2714    leucaemia myeloid agudaiccfractura did
                         ...                  
1601                                 pneumonia
3219                      iccinfec respiratory
1370                     sarcoidosis silicosis
899                               polimiositis
3004             insuficiencia cardiac chronic
Name: diagnostico, Length: 300, dtype: object

In [60]:
# Anteriormente teniamos el valor de ITU seguido de (infeccion del tracto urinario), para evitar paréntesis y más información simplificamos a ITU en los parámetros de "data_processing" para la columna diagnóstico
mask = df_sample['diagnostico'].str.contains(r'\(.*\)',regex=True)
filtered_values = df_sample[mask]

counts = filtered_values['diagnostico'].value_counts()
counts

Series([], Name: diagnostico, dtype: int64)

In [26]:
# TextBlob no es la panacea, devuelve muchos errores y traduce al inglés además de comerse palabras
from textblob import TextBlob

def correct_spelling(df):
    for column in df:
        if df[column].dtype == 'object':  # comprueba si la columna es de tipo 'object'
            df[column] = df[column].apply(lambda x: str(TextBlob(x).correct()))

In [64]:
%%time

correct_spelling(df_sample)

CPU times: total: 4min 42s
Wall time: 15min 10s


In [65]:
df_sample.head()

,h_procedencia,hospital_category,s_procedencia,procedencia_category,diagnostico,diagnosis_category,motivo_ing,ingreso_category,motivo_alta,alta_category,...,categorized_combined_otros,tiene_sedacion,morfina,midazolam,buscapina,haloperidol,levomepromazina,medico,ayuntamiento,year
117,clinics,Santiago,oncologia,Oncologia,cancer colon,Cancers y neoplasms,control sintomas,Sintomas,exist,Titus,...,Sintomas generals,0,0,0,0,0,0,sure,desconocido,2017
2768,onto,Santiago,pal,Unidad Paliativos,cancer colon,Cancers y neoplasms,control sintomas,Sintomas,exist,Titus,...,Desconocido/To especificado,1,0,0,0,0,0,villa del bastille,santiago,2021
374,no,no,no,Trot,tumor makar,Cancers y neoplasms,control sintomas,Sintomas,exist,Titus,...,Desconocido/To especificado,1,1,1,1,0,0,villa del bastille,desconocido,2017
2376,onto,Santiago,cirugia maxilofacial,Trot,cancer length studio of,Cancers y neoplasms,control lesion mandibular,Evaluaciones,in cuidados,Trot,...,Desconocido/To especificado,0,0,0,0,0,0,lope render,rose,2021
552,girl castres,Santiago,pal,Unidad Paliativos,cancer summon,Pulmonares y respiratorias,control door,Sintomas,exist,Titus,...,Complicaciones,1,1,1,1,0,1,bane also,desconocido,2017


## Conclusiones TextBlob

Text Blob solo para palabras en inglés.
No nos valdría para corregir ortografía en español...

# Spacy
* Textos con spacy

In [71]:
df_sample_2 = df.sample(300)

In [6]:
from spellchecker import SpellChecker
import spacy
from spacy.language import Language

spell = SpellChecker(language='es')  # especifica el idioma

@Language.component("spell_check")
def correct_spellings(doc):
    corrected_text = [spell.correction(word.text) if spell.correction(word.text) is not None else word.text for word in doc]
    return spacy.tokens.Doc(doc.vocab, words=corrected_text)

nlp = spacy.load('es_core_news_sm')  # carga el modelo de Spacy en español

nlp.add_pipe('spell_check', after='ner')

text = "Tus texto con errres"
doc = nlp(text)

print(doc.text)

def correct_text(text):
    doc = nlp(text)
    corrected_doc = correct_spellings(doc)
    return corrected_doc.text


Tus texto con errores 


In [72]:
%%time
df_sample_2['diagnostico_corregido'] = df_sample_2['diagnostico'].apply(correct_text)

CPU times: total: 40 s
Wall time: 1min 52s


In [ ]:
df_sample_2[['diagnostico','diagnostico_corregido']][:10]

,diagnostico,diagnostico_corregido
2106,cancer colon,cancer colon
2674,cancer colon,cancer colon
3698,ileostomia,ileostomia
2570,anciano fragil,anciano fragil
540,cancer pulmon estadio IV,cancer pulmon estadio IV
2557,neumonia,neumonia
3587,fistula aortoenterica,fístula aortoenterica
3975,upp sacra,up sacra
1431,bronquiectasias,bronquiectasias
2185,infeccion urinaria,infección urinaria


In [79]:
catalog.list("hado")

['hado_22',
 'hado_21',
 'hado_20',
 'hado_19',
 'hado_18',
 'hado_17',
 'strip_lower_hado_17',
 'strip_lower_hado_18',
 'strip_lower_hado_19',
 'strip_lower_hado_20',
 'strip_lower_hado_21',
 'strip_lower_hado_22',
 'hado_concat',
 'hado_clean',
 'hado_clean_na',
 'hado_replaced_words',
 'hado_cleaned',
 'hado_cleaned_sedation',
 'hado_cleaned_medication',
 'hado_categorized',
 'hado_final',
 'hado_encoded']

In [4]:
# Hemos probando con los datos en los que se ha realizado toda la limpieza
# Probaré con los datos sin los replaces pero sin NAs 'hado_clean'

df_clean = catalog.load('hado_clean_na')

[09/17/23 01:59:43] INFO     Loading data from 'hado_clean_na' (CSVDataSet)...                  data_catalog.py:345

In [8]:
df_clean['diagnostico_corregido'] = df_clean['diagnostico'].apply(correct_text)

In [9]:
df_clean[['diagnostico','diagnostico_corregido']][:10]

,diagnostico,diagnostico_corregido
0,ca orl,ca oro
1,melanoma vulvar,melanoma vulgar
2,ca broncogenico,ca broncogenico
3,cirrosis hepatica,cirrosis hepática
4,neplasia de mama estadio iv,neplasia de mama estadio iv
5,anemia,anemia
6,adenoca de pulmon,adenoma de pulmon
7,leucemia linfatica cronica,leucemia linfática cronica
8,fx meseta tibial,fe meseta tibial
9,cancer de mama,cancer de mama


## Conclusiones Spacy

No nos vale para corregir ortografía en español tampoco pero es mejor que TextBlob en resultados.



# Diccionarios

Para corregir los diagnosticos asi como otras columnas, optaremos por crear listas y diccionarios con las correcciones

In [10]:
corrections = {
    'iccbacteriemia': 'insuficiencia cardiaca cronica bacteriemia',
    'icc': 'insuficiencia cardiaca cronica',
    'ic': 'insuficiencia cardiaca',
    'insf ': 'insuficiencia',
    'i cardiaca': 'insuficiencia cardiaca',
    'icardiaca': 'insuficiencia cardiaca',
    'insufcardiaca': 'insuficiencia cardiaca',
    'insuficienciacardiaca': 'insuficiencia cardiaca',
    '1tu': 'ITU',
    'itu': 'ITU',
    'ituinfeccion': 'infeccion',
    'inf': 'infeccion',
    'infecc': 'infeccion',
    'infecion': 'infeccion',
    'infec': 'infeccion',
    'capulmon': 'cancer pulmon',
    'canc': 'cancer',
    'ca': 'cancer',
    'insuf': 'insuficiencia',
    'est': 'estadio',
    'e': 'estadio',
    'fr': 'fractura',
    'deteriorocognitivo': 'deterioro cognitivo',
    'iv': 'IV',
    'resp': 'respiratoria',
    '0steomielitis': 'osteomielitis',
    'neplasia': 'neoplasia',
    'adenoca': 'adenoma',
    'sepsisi': 'sepsis',
    'caepidermoide': 'carcinoma epidermoide',
    'ucera': 'ulcera',
    'alzheimertromboembolismo': 'alzheimer trombo embolismo',
    'deteriioro': 'deterioro',
    'miasteniabronquiectasias': 'miastenia bronquiectasia',
    'brnquiectasias': 'bronquiectasia',
    'quirurgicaarteriopatia': 'quirurgica arteriopatia',
    'respiratoriasuboclusion': 'respiratoria suboclusion',
    'respiratoriainsuf': 'respiratoria insuficiencia',
    'sistemicodeterioro': 'sistemico deterioro',
    'ituca': 'itu cancer',
    'ituincont': 'itu incontinencia',
    'ituinfec': 'itu infeccion',
    'piwe': 'pie',
    'pulm': 'pulmon',
    'urinarui': 'urinaria',
    'trtraparesia': 'tetraparesia',
    'colangiocarcinima': 'colangiocarcinoma',
    'bacteriemiadeterioro': 'bacteriemia deterioro',
    'wpidermoide': 'epidermoide',
    '(infeccion del tracto urinario)': 'ITU'
}

    # Añade tantas correcciones como necesites


In [13]:
# Busco todas las filas que contienen 'cancer' y las cuento
df_clean['diagnostico'][df_clean['diagnostico'].str.contains('cancer',case=False)].value_counts()

cancer de colon                  68
cancer de pulmon                 65
cancer de pancreas               26
cancer de prostata               18
cancer de mama                   17
                                 ..
cancer intestinal                 1
cancer de gastrico estadio iv     1
cancer de cardias                 1
cancer cavidad oral               1
cancer de recto fournier          1
Name: diagnostico, Length: 69, dtype: int64

In [18]:
# Solo palabras completas 'ca'
#\b es una aserción de límite de palabra en una expresión regular,
# lo que significa que 'ca' solo se encontrará si está rodeada de límites de palabra 
# (como espacios, signos de puntuación, el inicio del texto o el final del texto).


# Reemplaza los valores NaN con una cadena vacía
df_clean['diagnostico'] = df_clean['diagnostico'].fillna('')

df_clean['diagnostico'][df_clean['diagnostico'].str.contains(r'\bca\b', case=False, regex=True)].reset_index(drop=True).value_counts()

ca pancreas                     24
ca pulmon est iv                21
ca colon estadio iv             18
ca colon                        18
ca mama est iv                  16
                                ..
ca de pancreas                   1
ca de recto estiv                1
ca prostata ee iv                1
ca epidermoide suelo de boca     1
ca celulas de merkel             1
Name: diagnostico, Length: 198, dtype: int64

In [11]:
def correct_text(text):
    words = text.split()
    corrected_words = [corrections.get(word, word) for word in words]
    corrected_text = ' '.join(corrected_words)
    return corrected_text

In [28]:
correct_text('ca pulmon est iv')

'cancer pulmon estadio IV'

In [22]:
df_clean['diagnostico_corregido'][df_clean['diagnostico_corregido'].str.contains(r'\bca\b', case=False, regex=True)].reset_index(drop=True).value_counts()

ca pancreas                      24
ca pulmon est iv                 21
ca colon estadio iv              18
ca colon                         18
ca mama estadio iv               17
                                 ..
ca de recto esti                  1
ca prostata ee iv                 1
ca epidermoide suelo de boca      1
ca colon e iv                     1
ca celulas de merced              1
Name: diagnostico_corregido, Length: 195, dtype: int64

In [24]:

df['diagnostico'][df['diagnostico'].str.contains(r'\bca\b', case=False, regex=True)].reset_index(drop=True).value_counts()

Series([], Name: diagnostico, dtype: int64)

In [34]:
# Solo palabras completas 'ca'
df_cancer = df[df['diagnostico'].str.contains(r'\bcancer\b', case=False, regex=True)]

In [35]:
df_cancer.shape

(806, 50)

In [36]:
df_cancer['diagnostico'].value_counts()

cancer de pulmon            70
cancer de colon             70
cancer pulmon estadio IV    34
cancer mama estadio IV      33
cancer pancreas             30
                            ..
cancer de cardias            1
cancer vias biliar           1
cancer de orofarnge          1
cancer de tiroides           1
cancer celulas de merkel     1
Name: diagnostico, Length: 223, dtype: int64

<hr>

# Convertir los datos categóricos a un formato numérico:
Para convertir las variables categóricas en numéricas, puedes utilizar la función get_dummies() de pandas para realizar la codificación one-hot o la función LabelEncoder de la biblioteca scikit-learn para la codificación ordinal. Aquí tienes un ejemplo de cómo hacer la codificación one-hot:

In [39]:
df_cancer.columns

Index(['h_procedencia', 'hospital_category', 's_procedencia',
       'procedencia_category', 'diagnostico', 'diagnosis_category',
       'motivo_ing', 'ingreso_category', 'motivo_alta', 'alta_category',
       'fecha_alta', 'ap', 'n_estancias', 'n_visitas', 'paliativo_onc_noc',
       'paliativo_no_onc_noc', 'fiebre', 'disnea', 'dolor', 'delirium',
       'sedacion', 'p_terminal', 'agonia', 'ast_anorx', 'cronico_reag',
       'trato_antibiotico', 'transfusion', 'paracentesis', 'agudo_estable',
       'toracocentesis', 'fe_iv', 'ps_ecog', 'barthel', 'gds_fast', 'eva_ing',
       'otros_complicaciones', 'otros', 'otros_1', 'otros_2', 'otros_category',
       'categorized_combined_otros', 'tiene_sedacion', 'morfina', 'midazolam',
       'buscapina', 'haloperidol', 'levomepromazina', 'medico', 'ayuntamiento',
       'year'],
      dtype='object')

In [41]:
df_encoded = pd.get_dummies(df_cancer, columns=['diagnostico'])

In [42]:
df_encoded.head()

,h_procedencia,hospital_category,s_procedencia,procedencia_category,diagnosis_category,motivo_ing,ingreso_category,motivo_alta,alta_category,fecha_alta,...,diagnostico_infeccion urinaria cancer rinon,diagnostico_irc cancer colon estadio IV,diagnostico_itu cancer prostata hepatopatia cronica,diagnostico_itu incontinencia urinaria cancer cuello uterino,diagnostico_mediastinitis cancer pulmon,diagnostico_neumonia cancer colon,diagnostico_recidiva cancer esofago,diagnostico_sospecha cancer tiroides,diagnostico_sospecha de cancer rectosigmoideo,diagnostico_vomitos y diarrea cancer colon estadio IV
0,gil casares,Santiago,upal,Unidad Paliativos,Canceres y neoplasias,control sintomas,Sintomas,reingreso,Complicaciones,unknown,...,0,0,0,0,0,0,0,0,0,0
2,clinico,Santiago,mir,MIR,Canceres y neoplasias,control evolutivo,Evaluaciones,fin cuidados,Otros,unknown,...,0,0,0,0,0,0,0,0,0,0
9,clinico,Santiago,urgencias,Urgencias,Canceres y neoplasias,control sintomas,Sintomas,exitus,Exitus,unknown,...,0,0,0,0,0,0,0,0,0,0
10,clinico,Santiago,oncologia,Oncologia,Canceres y neoplasias,dolor,Sintomas,fin cuidados,Otros,unknown,...,0,0,0,0,0,0,0,0,0,0
11,clinico,Santiago,oncologia,Oncologia,Canceres y neoplasias,dolor,Sintomas,fin cuidados,Otros,unknown,...,0,0,0,0,0,0,0,0,0,0


<hr>

#  Normalizar o estandarizar las variables numéricas:
Para normalizar o estandarizar las variables numéricas, puedes utilizar las funciones MinMaxScaler o StandardScaler de scikit-learn. Aquí tienes un ejemplo de cómo estandarizar una columna numérica:

In [46]:
from sklearn.preprocessing import StandardScaler

df_cancer_num = df_cancer.select_dtypes(exclude='object')

scaler = StandardScaler()
for col in df_cancer_num:
    df_cancer_num[col] = scaler.fit_transform(df_cancer_num[[col]])

In [48]:
df_cancer_num.describe().T

,count,mean,std,min,25%,50%,75%,max
n_estancias,806.0,-2.644700e-17,1.000621,-0.873505,-0.622973,-0.347388,0.228836,6.842884
n_visitas,806.0,-1.763133e-17,1.000621,-0.934868,-0.567066,-0.321865,0.168538,7.769775
ps_ecog,806.0,2.027603e-16,1.000621,-2.057558,-0.231471,0.377225,0.377225,22.898971
barthel,806.0,-3.526267e-17,1.000621,-0.460188,-0.446008,-0.431829,0.106998,6.714717
gds_fast,806.0,-1.763133e-17,1.000621,-0.181376,-0.181376,-0.181376,-0.181376,7.946671
eva_ing,806.0,0.000000e+00,1.000621,-0.320407,-0.320407,-0.320407,-0.320407,4.598606
tiene_sedacion,806.0,-4.407833e-18,1.000621,-0.668609,-0.668609,-0.668609,1.495643,1.495643
morfina,806.0,-7.052533e-17,1.000621,-0.222439,-0.222439,-0.222439,-0.222439,4.495612
midazolam,806.0,-7.052533e-17,1.000621,-0.222439,-0.222439,-0.222439,-0.222439,4.495612
buscapina,806.0,-3.526267e-17,1.000621,-0.213062,-0.213062,-0.213062,-0.213062,4.693460


<hr>

# Verificar si hay errores en los datos y corregirlos:
Durante el análisis exploratorio de datos (EDA), presta atención a las distribuciones y valores atípicos. Si encuentras errores, corrige los datos según sea necesario.

<hr>

# Comprobar si hay duplicados y decidir cómo tratarlos:
Puedes utilizar la función duplicated() de pandas para encontrar registros duplicados y la función drop_duplicates() para eliminarlos. Aquí tienes un ejemplo de cómo eliminar duplicados:

In [54]:
df.drop_duplicates(inplace=True)